In [ ]:
import gather_memtier_statistics as gmts
import graph_xputresp_vs_multigets as xput_vs_multi
import graph_xput_vs_time as xput_vs_time
import graph_responsetime as resptime
import graph_queuelength as queuelen
import summary_table as summ
import matplotlib.pyplot as plt
import numpy as np
import os, sys

In [ ]:
exp5_1_dir = "" # 5.1 sharded experiment dir
exp5_2_dir = "" # 5.2 nonsharded experiment dir
outdir = "./graphs"

In [ ]:
# Experiment 5: MultiGets
# Plot the throughput and response time measured in the middleware

experiment_label = "exp5"

middlewares = ["middleware_04", "middleware_05"]
client_logfiles = ["client_01_0.log", "client_01_1.log", "client_02_0.log", "client_02_1.log", "client_03_0.log", "client_03_1.log"]
dir_suffix_regex_string = "_\d*multiget"
warmup_period_endtime = 10
cooldown_period_starttime = 72
outdir = "./graphs"
num_threads = 6
ylim_xput = 18000
ylim_resp = 2.5
xlim = 10
shard = xput_vs_multi.gather_data(exp5_1_dir, experiment_label, "sharded", middlewares, client_logfiles, dir_suffix_regex_string, warmup_period_endtime, cooldown_period_starttime, outdir, num_threads, ylim_xput, ylim_resp, xlim)
nonshard = xput_vs_multi.gather_data(exp5_2_dir, experiment_label, "nonsharded", middlewares, client_logfiles, dir_suffix_regex_string, warmup_period_endtime, cooldown_period_starttime, outdir, num_threads, ylim_xput, ylim_resp, xlim)

shard = shard.sort_values('multigetSize')


nonshard = nonshard.sort_values('multigetSize')


In [ ]:
## Response Time

means_shard_resp = shard.loc['mean', 'responsetime']
std_shard_resp = shard.loc['std', 'responsetime']

means_nonshard_resp = nonshard.loc['mean', 'responsetime']
std_nonshard_resp = nonshard.loc['std', 'responsetime']

indices = np.arange(4)
width = 0.35 
fig, ax = plt.subplots()
shard_bar = ax.bar(indices, means_shard_resp, width, color='#e31a1c', yerr=std_shard_resp)
nonshard_bar = ax.bar(indices + width, means_nonshard_resp, width, color='#fed976', yerr=std_nonshard_resp)

ax.legend((shard_bar[0], nonshard_bar[0]), ('Sharded', 'Non-Sharded'), loc='upper left')
ax.set_xticks(indices + width / 2)
ax.set_xticklabels(('1', '3', '6', '9'))
ax.set_ylabel('GET Response Time (ms)')
ax.set_xlabel('Number of keys in GETs')
ax.set_title('Response Time (MT) vs. different MultiGET sizes')
ax.set_ylim([0,2.5])

plt.show()
fig.savefig(os.path.join(outdir, experiment_label, "responseTime_multigets.png"), dpi=300)

In [ ]:
## Request Throughput

means_shard_xput = shard.loc['mean', 'throughput']
std_shard_xput = shard.loc['std', 'throughput']

means_nonshard_xput = nonshard.loc['mean', 'throughput']
std_nonshard_xput = nonshard.loc['std', 'throughput']

indices = np.arange(4)
width = 0.35 
fig, ax = plt.subplots()
shard_bar = ax.bar(indices, means_shard_xput, width, color='#006d2c', yerr=std_shard_xput)
nonshard_bar = ax.bar(indices + width, means_nonshard_xput, width, color='#66c2a4', yerr=std_nonshard_xput)

ax.legend((shard_bar[0], nonshard_bar[0]), ('Sharded', 'Non-Sharded'), loc='upper right')
ax.set_xticks(indices + width / 2)
ax.set_xticklabels(('1', '3', '6', '9'))
ax.set_ylabel('Throughput (op/s)')
ax.set_xlabel('Number of keys in GETs')
ax.set_title('Throughput (MT) vs. different MultiGET sizes')
ax.set_ylim([0,11000])

plt.show()
fig.savefig(os.path.join(outdir, experiment_label, "throughput_multigets.png"), dpi=300)

In [ ]:
 ## Key Throughput

shard['key_throughput'] = shard['throughput'] * shard['multigetSize']
nonshard['key_throughput'] = nonshard['throughput'] * nonshard['multigetSize']

means_shard_xput = shard.loc['mean', 'key_throughput']
std_shard_xput = shard.loc['std', 'key_throughput']

means_nonshard_xput = nonshard.loc['mean', 'key_throughput']
std_nonshard_xput = nonshard.loc['std', 'key_throughput']

indices = np.arange(4)
width = 0.35 
fig, ax = plt.subplots()
shard_bar = ax.bar(indices, means_shard_xput, width, color='#4d004b', yerr=std_shard_xput)
nonshard_bar = ax.bar(indices + width, means_nonshard_xput, width, color='#88419d', yerr=std_nonshard_xput)

ax.legend((shard_bar[0], nonshard_bar[0]), ('Sharded', 'Non-Sharded'), loc='upper left')
ax.set_xticks(indices + width / 2)
ax.set_xticklabels(('1', '3', '6', '9'))
ax.set_ylabel('Throughput (keys/s)')
ax.set_xlabel('Number of keys in GETs')
ax.set_title('Key Throughput (MT) vs. different MultiGET sizes')
ax.set_ylim([0,70000])

plt.show()
fig.savefig(os.path.join(outdir, experiment_label, "key_throughput_multigets.png"), dpi=300)

In [ ]:
## Drawing the response time break-down of a a few interesting configurations of the sharded experiment
reps = [1, 2, 3]
middlewares = ["middleware_04", "middleware_05"]
client_logfiles = ["client_01_0.log", "client_01_1.log", "client_02_0.log", "client_02_1.log", "client_03_0.log", "client_03_1.log"]
warmup_period_endtime = 10
cooldown_period_starttime = 72
outdir = "./graphs"
experiment_label = "exp5"
xlim=2

fig, axes = plt.subplots(1,4, sharey=True, figsize=(15, 4))
resptime.graph_responsetime(1, "sharded", middlewares, client_logfiles, reps, exp5_1_dir, xlim, axes[0])
resptime.graph_responsetime(3, "sharded", middlewares, client_logfiles, reps, exp5_1_dir, xlim, axes[1])

resptime.graph_responsetime(6, "sharded", middlewares, client_logfiles, reps, exp5_1_dir, xlim, axes[2])
resptime.graph_responsetime(9, "sharded", middlewares, client_logfiles, reps, exp5_1_dir, xlim, axes[3])
fig.tight_layout()
plt.show()

fig.savefig(os.path.join(outdir, experiment_label, "responseTimeBreakDown_sharded.png"), dpi=300)